In [68]:
from ase.visualize import view
import ase
from ase.build import fcc111, bcc111
import numpy as np
import cv2

In [205]:
slabA = fcc111('Al', size=(8,8,5))
slabB = bcc111('Fe', size=(8,8,5))
print(slabA)

view(slabA)
view(slabB)

Atoms(symbols='Al320', pbc=[True, True, False], cell=[[22.91025971044414, 0.0, 0.0], [11.45512985522207, 19.84086691654374, 0.0], [0.0, 0.0, 0.0]], tags=...)


In [212]:
xlen, ylen, zlen = 8,8,8
unit = np.array([1,1,1])



atoms = np.zeros((xlen * ylen * zlen, 3))

for x in range(xlen):
    for y in range(ylen):
        for z in range(zlen):
            i = x + y*xlen + z*xlen*ylen
            atoms[i] = [x, y, z]
print('atoms', atoms.shape)

unit_norm = unit / np.linalg.norm(unit)
print(unit, unit_norm)

dists = atoms @ unit 
dists -= np.sum(dists) / xlen/ylen/zlen
print('dists', dists.shape)
projs = atoms - np.outer(dists, unit_norm)

ordered_dists = np.sort(dists)
maxd = np.max(np.abs(ordered_dists[:-1] - ordered_dists[1:]))
print('maxd', maxd)

# print(dists)
mod_dists = dists % (maxd*2)
pruned_dists = []
pruned_projs = []
pruned_atoms = []
def pcopy(i, proj):
    for pruned_proj in pruned_projs:
        if np.sum(np.abs(proj - pruned_proj)) < 0.0001:
            return
    pruned_dists.append(mod_dists[i])
    pruned_projs.append(proj)
    pruned_atoms.append(atoms[i])

for i, p in enumerate(projs):
    pcopy(i, p)

pruned_dists = np.array(pruned_dists)
pruned_projs = np.array(pruned_projs)
pruned_atoms = np.array(pruned_atoms, dtype=int)


grid = np.zeros((np.max(pruned_atoms[:, 0])+1, np.max(pruned_atoms[:, 1])+1))
for i, a in enumerate(pruned_atoms):
    x, y, _ = a.astype(int)
    img[x, y] = pruned_dists[i]

atoms (512, 3)
[1 1 1] [0.57735027 0.57735027 0.57735027]
dists (512,)
maxd 1.0


array([[0.5, 1.5, 0.5, 1.5, 0.5, 1.5, 0.5, 1.5],
       [1.5, 0.5, 1.5, 0.5, 1.5, 0.5, 1.5, 0.5],
       [0.5, 1.5, 0.5, 1.5, 0.5, 1.5, 0.5, 1.5],
       [1.5, 0.5, 1.5, 0.5, 1.5, 0.5, 1.5, 0.5],
       [0.5, 1.5, 0.5, 1.5, 0.5, 1.5, 0.5, 1.5],
       [1.5, 0.5, 1.5, 0.5, 1.5, 0.5, 1.5, 0.5],
       [0.5, 1.5, 0.5, 1.5, 0.5, 1.5, 0.5, 1.5],
       [1.5, 0.5, 1.5, 0.5, 1.5, 0.5, 1.5, 0.5]])

In [215]:
img = np.zeros((400, 400, 3))
cv2.imshow('img', img)

In [209]:
a = np.array([[x, x, x] for x in range(3)])
print(a.shape, atoms.T.shape)
view(ase.Atoms('N3', positions=atoms[:3].T))
view(ase.Atoms('N3', positions=a))


(3, 3) (3, 512)
